## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_mutual_info_score
import pandas as pd

### Importando o dataset

In [2]:
df = pd.read_csv('/workspaces/Fuzzy_Clustering/datasets/seeds.csv')
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


### Verificando as classes

In [3]:
df["V8"].unique()

array([1, 2, 3])

### Verificando o nome exato das colunas

In [4]:
df.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8'], dtype='object')

### Retirando variáveis inúteis e transformando classes em números

In [5]:
df = df.rename(columns={'V8': 'Class'})
df["Class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
df.head()

/tmp/ipykernel_19096/411401851.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)


,V1,V2,V3,V4,V5,V6,V7,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0


### Verificando a corretude das classes

In [6]:
df["Class"].unique()

array([0, 1, 2])

### Armazenando as classes em uma variável separada

In [7]:
labels = df["Class"].values
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
df.drop("Class", axis=1, inplace=True)
df.head()

,V1,V2,V3,V4,V5,V6,V7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


### Retirando a classe para isolar as variáveis

In [9]:
dados = df.to_numpy()
dados

array([[15.26  , 14.84  ,  0.871 , ...,  3.312 ,  2.221 ,  5.22  ],
       [14.88  , 14.57  ,  0.8811, ...,  3.333 ,  1.018 ,  4.956 ],
       [14.29  , 14.09  ,  0.905 , ...,  3.337 ,  2.699 ,  4.825 ],
       ...,
       [13.2   , 13.66  ,  0.8883, ...,  3.232 ,  8.315 ,  5.056 ],
       [11.84  , 13.21  ,  0.8521, ...,  2.836 ,  3.598 ,  5.044 ],
       [12.3   , 13.34  ,  0.8684, ...,  2.974 ,  5.637 ,  5.063 ]],
      shape=(210, 7))

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik} = 1$ para todo $k \in \Omega$.

In [10]:
def inicializao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Atualização dos centroides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

### $y_i = \frac{\sum_{k=1}^n(u_{ik})^mx_k}{\sum_{k=1}^n(u_{ik})^m}$

In [11]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

### $u_{ik}[\sum_{h=1}^c\{\frac{d(x_k,y_i)}{d(x_k,y_h)}\}^\frac{1}{m-1}]^{-1}$

onde

$d(x_k,y_i) = \sum_{j=1}^p(x_k^j-y_i^j)^2$

In [12]:
def atualizacao_matriz_pertinencia(dados, centroides, m): # talvez a fórmula para calcular a matriz de pertinência esteja errada
    # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
    # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
    # ** 2 para a distância ser a quadrada
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

### Fuzzy C-Means

Ações:
1. Inicialização da matriz de pertinência
2. Atualização dos centroides
3. Atualização da matriz de pertinência

Critérios de parada:
1. Número máximo de iterações atingido
2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

In [13]:
def fcm(dados, num_clusters, m=2, max_iter=1000, erro=1e-5):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia

### Índice de Rand Ajustado (IRA)

In [14]:
def indice_ami(labels, predicted_labels):
    return adjusted_mutual_info_score(labels, predicted_labels)

### Simulação de Monte Carlo

In [15]:
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        idx_ami = indice_ami(labels, predicted_labels)
        indices_rand.append(idx_ami)
    mean_ami = np.mean(indices_rand)
    std_ami = np.std(indices_rand)
    return mean_ami, std_ami

### Definição de parâmetros e execução do método

In [16]:
num_clusters = 3
num_trials = 100
media_indice_ami, dp_indice_ami = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_ami:.4f}")
print(f"Standard Deviation of Rand Index: {dp_indice_ami:.4f}")

Monte Carlo FCM Clustering Results (100 trials)
Mean Rand Index: 0.6922
Standard Deviation of Rand Index: 0.0000
